# No Man's Sky Updates Analysis

## Table of Contents
1. [Introduction](#Introduction)
2. [Data Description](#Data-Description)

## Introduction
The purpose of this analysis is to:
- Examine the frequency and patterns of updates for No Man's Sky.
- Provide insights into the development cycle.
- Help predict future updates.
- Identify periods of high and low activity for planning game events or releases.

## Data Description
The dataset includes:
- Dates and descriptions of each update.
- Data collected from official No Man's Sky update logs.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import numpy as np

# Load the update data for No Man's Sky
noman_update_data = pd.read_csv('../data/Noman/noman_updates_collection.csv')

# Display the first few rows of the data
print(noman_update_data.head())

  Category                                              Title  \
0    patch                 No Man's Sky Companions Patch 3.21   
1    patch  No Man's Sky Update 3.2 patch notes bring adop...   
2    patch                               Crossplay Patch 2.52   
3    patch                                Exo Mech Patch 2.42   
4    patch  No Man's Sky Beyond patch notes: VR, Multiplay...   

                  Date  
0  2021-02-19 14:45:50  
1  2021-02-17 07:40:00  
2  2020-06-12 16:25:20  
3  2020-04-16 13:50:00  
4  2019-08-15 15:57:34  


In [7]:
import re

# Convert the 'Date' column to datetime
noman_update_data['Date'] = pd.to_datetime(noman_update_data['Date'])

# Define a function to extract version numbers
def extract_version(title):
    match = re.search(r'(\d+\.\d+)', title)
    if match:
        return match.group(0)
    return None

# Apply the function to extract versions
noman_update_data['Version'] = noman_update_data['Title'].apply(extract_version)

# Drop rows with no version information
noman_update_data = noman_update_data.dropna(subset=['Version']).copy()

# Extract major and minor version numbers
noman_update_data.loc[:, 'Major_Version'] = noman_update_data['Version'].str.split('.').str[0].astype(float)
noman_update_data.loc[:, 'Minor_Version'] = noman_update_data['Version'].str.split('.').str[1].astype(float)

# Create a combined version number
noman_update_data.loc[:, 'Combined_Version'] = noman_update_data['Major_Version'] + noman_update_data['Minor_Version'] / 100

# Sort the dataframe by date
noman_update_data = noman_update_data.sort_values('Date')

# Reset the index
noman_update_data = noman_update_data.reset_index(drop=True)

print(noman_update_data.head())
print(noman_update_data.tail())

  Category             Title                Date Version  Major_Version  \
0    patch  Patch Notes 1.07 2016-09-02 17:25:14    1.07            1.0   
1    patch  Patch Notes 1.07 2016-09-02 17:25:14    1.07            1.0   
2    patch        Patch 1.12 2016-12-07 10:54:35    1.12            1.0   
3    patch        Patch 1.12 2016-12-07 10:54:35    1.12            1.0   
4    patch        Patch 1.13 2016-12-12 12:00:10    1.13            1.0   

   Minor_Version  Combined_Version  
0            7.0              1.07  
1            7.0              1.07  
2           12.0              1.12  
3           12.0              1.12  
4           13.0              1.13  
   Category                                              Title  \
47   update  No Man's Sky focuses on customising and access...   
48   update  No Man's Sky update 4.0, Waypoint, caps off th...   
49   update  No Man's Sky' 4.1 Fractal Update adds an Exped...   
50   update  No Man's Sky goes 5.0 with a sweeping tech upd... 